The file `serra.importation_status` contains recombination coordinates for all sequences in the alignment. First we merge the coordinates across all sequences (bedtoold merge), then we remove the given nucleotides from all sequences.

In [11]:
import pandas as pd
from Bio import AlignIO

In [3]:
rec = pd.read_csv('../clonal/serra.importation_status.txt', sep='\t')

In [4]:
rec

,Node,Beg,End
0,GCA_946228595.1_26968_7_169,925165,928546
1,GCA_946228595.1_26968_7_169,1156205,1157164
2,GCA_946228595.1_26968_7_169,1170106,1170336
3,GCA_946228595.1_26968_7_169,1421573,1423897
4,GCA_946228595.1_26968_7_169,1551512,1552398
...,...,...,...
130177,NODE_3711,1024909,1024978
130178,NODE_3711,1136070,1136085
130179,NODE_3711,1180503,1180536
130180,NODE_3711,1518079,1518529


In [5]:
rec.Node = 1     # So it's as if all the sequences are on the same chromosome, so they will be merged

In [6]:
rec_sorted = rec.sort_values(by=['Beg', 'End']) # "merge" requires that the input is sorted 
                                                # by chromosome and then by start coordinate

In [7]:
rec_sorted.to_csv('../clonal/recomb.bed',index=False, header=False, sep='\t')

Running commands from the command line:  

```
bedtools merge -i recomb.bed > recomb_merged.txt
cat recomb_merged.txt | wc -l
``` 

All recombinations merged in 479 ranges

In [9]:
recomb_merged = pd.read_csv('../clonal/recomb_merged.txt', sep='\t', header=None)

In [10]:
recomb_merged

,0,1,2
0,1,1,481
1,1,521,772
2,1,793,2660
3,1,2842,3868
4,1,4066,9004
...,...,...,...
474,1,1745701,1751665
475,1,1752026,1753621
476,1,1753639,1756022
477,1,1756139,1766201


In [12]:
alignment = AlignIO.read(open("../core_gene_alignment_copy.fixed.fasta"), "fasta")
print("Alignment length %i" % alignment.get_alignment_length())

Alignment length 1768179


In [13]:
for record in alignment:
    for i in range(len(recomb_merged)-1,-1,-1):
        s = recomb_merged.loc[:,1][i]-1     # s - start   (index in python -1)
        e = recomb_merged.loc[:,2][i]                #  without -1 - not include end coordinate  
        record.seq = record.seq[:s] + record.seq[e:]


In [ ]:
print("Alignment length %i" % alignment.get_alignment_length())

In [ ]:
AlignIO.write(alignment, '../clonal/align_wo_rec.fasta', 'fasta')